In [ ]:
#라이브러리 import

import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Max Time-windows in seconds
# 8 Hours x 60 x 60 = 28800 seconds
max_tw = 28800

In [ ]:
# 각 절단장비에 대한 배열 선언
arr_w14m1 = []
arr_w14m2 = []

arr_w21m1 = []
arr_w21m2 = []
arr_w21m3 = []

arr_w22m1 = []
arr_w22m3 = []
arr_w22m4 = []

arr_w23m1 = []
arr_w23m3 = []
arr_w23m4 = []

arr_w24m1 = []
arr_w24m2 = []
arr_w24m3 = []

arr_w25m2 = []
arr_w25m3 = []
arr_w25m4 = []

arr_w26m1 = []
arr_w26m2 = []

arr_w27m1 = []
arr_w27m2 = []
arr_w27m3 = []

arr_w91m1 = []

arr_w92m1 = []
arr_w92m2 = []

arr_w96m1 = []
arr_w96m3 = []

arr_w97m3 = []
arr_w97m4 = []

In [ ]:
#각 작업장에 있는 절단장비 배열
arr_wm = [arr_w14m1, arr_w14m2, arr_w21m1, arr_w21m2, arr_w21m3, arr_w22m1, arr_w22m3, arr_w22m4, arr_w23m1, arr_w23m3,
          arr_w23m4, arr_w24m1, arr_w24m2, arr_w24m3, arr_w25m2, arr_w25m3, arr_w25m4, arr_w26m1, arr_w26m2, arr_w27m1,
          arr_w27m2, arr_w27m3, arr_w91m1, arr_w92m1, arr_w92m2, arr_w96m1, arr_w96m3, arr_w97m3, arr_w97m4]

#절단장비의 이름 배열
arr_mch_name = ['mch-141', 'mch-142', 'mch-211', 'mch-212', 'mch-213','mch-221', 'mch-223', 'mch-224', 'mch-231',
                'mch-233', 'mch-234', 'mch-241', 'mch-242', 'mch-243', 'mch-252', 'mch-253', 'mch-254', 'mch-261',
                'mch-262', 'mch-271', 'mch-272', 'mch-273', 'mch-911', 'mch-921', 'mch-922', 'mch-961', 'mch-963',
                'mch-973', 'mch-974']

In [ ]:
# 각 절단장비의 Remaining Time 배열 선언 이후 최대로 우선 설정
arr_mch_rtslot = []
for i in range(29):
    arr_mch_rtslot.append(max_tw)

In [ ]:
# Taken Job Index : 배치된 작업의 index
arr_tji = []

In [ ]:
#데이터 loading

ori_job = data_marking['예측 작업시간']
ori_job 

0         1477.58
1         3244.89
2         1192.87
3         4967.20
4         1975.54
           ...   
144402    3699.91
144403    2104.45
144404    1119.96
144405    1109.60
144406    1061.05
Name: 예측 작업시간, Length: 144407, dtype: float64

In [ ]:
job = copy.deepcopy(ori_job)

In [ ]:
#배치된 작업을 체크해주는 함수

def check_fit_jobs():
    fit_status = False
    fit_job_counter = 0

    # Upper & Lower Bound of RTS
    num_of_tj = len(arr_tji)

    lb_rts = 0
    if num_of_tj > 0:
        up_rts = max(arr_mch_rtslot)
    else:
        up_rts = max_tw

    num_of_job = len(job)
    for i in range(num_of_job):
        tmtct = job[i]
        if lb_rts < tmtct <= up_rts:
            fit_job_counter += 1

    if fit_job_counter > 0:
        fit_status = True

    return fit_status

In [ ]:
#remaining Time이 가장 큰 작업장의 인덱스를 고르는 함수
def get_max_mrtslot():
    # mch_idx = arr_mch_rtslot.index(max(arr_rtslot))
    arr_possible_mch = []
    num_of_mch = len(arr_wm)
    temp_max_rtslot = 0

    for i in range(num_of_mch):
        if arr_mch_rtslot[i] >= temp_max_rtslot:
            arr_possible_mch.append(i)

            temp_max_rtslot = arr_mch_rtslot[i]

    # Select random machine
    numof_possible_mch = len(arr_possible_mch)

    if numof_possible_mch > 0:
        sel_idx = np.random.randint(0, numof_possible_mch)
        sel_mch_idx = arr_possible_mch[sel_idx]

    return sel_mch_idx

In [ ]:
#활용할 수 있는 작업의 인덱스를 고르는 함수
def get_avail_job(mch_idx):
    arr_avail_jidx = []
    num_of_job = len(job)

    # Get rtslot of selected machine
    mch_rtslot = arr_mch_rtslot[mch_idx]

    for i in range(num_of_job):
        # AVOIDED Taken Job
        if job[i] > 0:
            # if total MTCT of current job less than the remaining time slot
            # of selected machine
            if job[i] <= mch_rtslot:
                arr_avail_jidx.append(i)

    return arr_avail_jidx

In [ ]:
#활용할 수 있는 작업 인덱스들 중 하나의 작업을 랜덤하게 선택하는 함수
def select_random_job(arr_avail_jidx):
    numof_avail_job = len(arr_avail_jidx)

    sel_idx = np.random.randint(0, numof_avail_job)
    sel_job_idx = arr_avail_jidx[sel_idx]

    return sel_job_idx

In [ ]:
#선택된 작업을 선택된 작업장의 절단장비에 할당하는 함수
def assign_job_2machine(job_idx, mch_idx):
    # Assign selected job to selected machine
    arr_wm[mch_idx].append(job_idx)

    # Update the remaining time slot of the selected machine
    arr_mch_rtslot[mch_idx] = arr_mch_rtslot[mch_idx] - job[job_idx]

    # Set the selected job value to 0
    job[job_idx] = 0

    # Flagging the selected job as taken job
    arr_tji.append(job_idx)

In [ ]:
# ********************************** #
# Main Program
# ********************************** #
fj = check_fit_jobs()

In [ ]:
#함수를 이용하여 배치
while fj:
    # Select machine with maximum remaining time slot
    sel_mch_idx = get_max_mrtslot()

    # Find available jobs that fit with remaining time slot of selected machine
    arr_avail_jidx = get_avail_job(sel_mch_idx)

    # Make sure if there is a job that fit with remaining time slot of selected machine
    while len(arr_avail_jidx) == 0:
        sel_mch_idx = get_max_mrtslot()
        arr_avail_jidx = get_avail_job(sel_mch_idx)

    # Select random job
    if len(arr_avail_jidx) > 0:
        sel_job_idx = select_random_job(arr_avail_jidx)

    #Recalculate Selected the job



    # Assign a Job to a Machine
    assign_job_2machine(sel_job_idx, sel_mch_idx)

    fj = check_fit_jobs()

In [ ]:
# 결과 프린트
num_of_mch = len(arr_wm)
for i in range(num_of_mch):
    job_len = len(arr_wm[i])

    print("----------------------------------------")
    print("Job Index Processing in Machine ", arr_mch_name[i])
    print("----------------------------------------")
    print(arr_wm[i])
    print()